In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from skbio.diversity import beta_diversity
from skbio.stats.ordination import pcoa

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

In [ ]:
data_dir = "../outputs_rrndb/everything"

In [ ]:
def _load(ab_path: str, metadata_path: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    meta = pd.read_csv(metadata_path, index_col=0)
    df = pd.read_csv(ab_path, index_col=0)
    drop_samples = df.index.str.contains("-V-") | df.index.str.contains("-SNS-")
    # drop_samples = df.index.str.contains("-5N-")
    df = df.loc[~drop_samples]
    meta = meta.loc[~drop_samples]
    nonzero = df.sum(axis=1).astype(bool)
    df = df.loc[nonzero]
    meta = meta.loc[nonzero]

    source = []
    for i in meta.index:
        if "-S-" in i:
            if "-OGAmp2-" in i:
                source.append("Swab-new")
            elif "-OG-" in i:
                source.append("Swab-old")
            elif "-OGcomb-" in i:
                source.append("Swab-comb")
            else:
                raise ValueError(i)
        if "-V-" in i:
            source.append("Vortex")
        if "-SNS-" in i:
            source.append("Swab no spike-in")
        if "-R-" in i:
            source.append("R2A")
        if "-T-" in i:
            source.append("TSA")
    meta["source"] = source
    
    print(np.unique([i.split("-")[2] for i in meta.index]))
    rep1, rep2 = np.unique([i.split("-")[2] for i in meta.index]).reshape(-1, 2).T
    print(rep1, rep2)
    rep_number = []
    for i in meta.index:
        if any(f"-{r}-" in i for r in rep1):
            rep_number.append("rep 1")
        elif any(f"-{r}-" in i for r in rep2):
            rep_number.append("rep 2")
        else:
            raise ValueError(i)
    meta["rep"] = rep_number
    return df, meta

## sample x OTU count PCA

In [ ]:
count_sample_otu, meta = _load(
    f"{data_dir}/count_sample_otu.csv", f"{data_dir}/metadata_sample.csv"
)

In [ ]:
pc = pcoa(beta_diversity("braycurtis", count_sample_otu), number_of_dimensions=10).samples.copy()
pc.index = count_sample_otu.index
pc[["sample_group", "source"]] = meta[["sample_group", "source"]]

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
sns.scatterplot(data=pc, x="PC1", y="PC2", hue="sample_group", style="source", ax=ax)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

fig, ax = plt.subplots(figsize=(5, 5))
sns.scatterplot(data=pc, x="PC2", y="PC3", hue="sample_group", style="source", ax=ax)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

## Sample x OTU log10 relative abundance PCA

In [ ]:
rel_ab_sample_otu, meta = _load(
    f"{data_dir}/rel_ab_sample_otu.csv", f"{data_dir}/metadata_sample.csv"
)
rel_ab_sample_otu = np.log10(rel_ab_sample_otu + 1e-8)

In [ ]:
pc_obj = PCA(n_components=10).fit(rel_ab_sample_otu)
pc = pc_obj.transform(rel_ab_sample_otu)
variance = pc_obj.explained_variance_ratio_
pc = pd.DataFrame(
    pc,
    index=rel_ab_sample_otu.index,
    columns=[f"PC{i}" for i in range(1, pc.shape[1] + 1)],
)
pc[["sample_group", "source"]] = meta[["sample_group", "source"]]

In [ ]:
pc_dims = 2, 3
fig, ax = plt.subplots(figsize=(5, 5))
sns.scatterplot(
    data=pc,
    x=f"PC{pc_dims[0]}",
    y=f"PC{pc_dims[1]}",
    hue="sample_group",
    style="source",
    ax=ax,
)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
ax.set_xlabel(f"PC{pc_dims[0]} ({variance[pc_dims[0] - 1]:.2f})")
ax.set_ylabel(f"PC{pc_dims[1]} ({variance[pc_dims[1] - 1]:.2f})")

## Sample x Genus log10 relative abundance PCA

In [ ]:
rel_ab_sample_genus, meta = _load(
    f"{data_dir}/rel_ab_sample_genus.csv", f"{data_dir}/metadata_sample.csv"
)
rel_ab_sample_genus = np.log10(rel_ab_sample_genus + 1e-8)

pc_obj = PCA(n_components=10).fit(rel_ab_sample_genus)
pc = pc_obj.transform(rel_ab_sample_genus)
variance = pc_obj.explained_variance_ratio_
pc = pd.DataFrame(
    pc,
    index=rel_ab_sample_genus.index,
    columns=[f"PC{i}" for i in range(1, pc.shape[1] + 1)],
)
pc[["Sample group", "Source", "Replication number"]] = meta[
    ["sample_group", "source", "rep"]
]

pc_dims = 2, 3
fig, ax = plt.subplots(figsize=(5, 5))
# get n colors where n is the number of unique values in the `sample_group` column
n = len(pc["Sample group"].unique())
row_colors = sns.color_palette("tab10", n)
sns.scatterplot(
    data=pc,
    x=f"PC{pc_dims[0]}",
    y=f"PC{pc_dims[1]}",
    hue="Sample group",
    palette=row_colors,
    style="Source",
    ax=ax,
)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)


# add ellipses
def eigsorted(cov):
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    return vals[order], vecs[:, order]

for group in pc["Sample group"].unique():
    x = pc.loc[pc["Sample group"] == group, f"PC{pc_dims[0]}"]
    y = pc.loc[pc["Sample group"] == group, f"PC{pc_dims[1]}"]
    mean_x, mean_y = x.mean(), y.mean()
    cov = np.cov(x, y)
    lambda_, v = eigsorted(cov)
    theta = np.degrees(np.arctan2(*v[:, 0][::-1]))
    w, h = 2 * 2 * np.sqrt(lambda_)
    ell = Ellipse(
        xy=(mean_x, mean_y),
        width=w,
        height=h,
        angle=theta,
        # color=row_colors[pc["sample_group"].unique().tolist().index(group)],
        alpha=0.2,
    )
    ell.set_facecolor(row_colors[pc["Sample group"].unique().tolist().index(group)])
    ell.set_edgecolor("grey")
    ax.add_artist(ell)

ax.set_xlabel(f"PC{pc_dims[0]} ({variance[pc_dims[0] - 1] * 100:.2f}%)")
ax.set_ylabel(f"PC{pc_dims[1]} ({variance[pc_dims[1] - 1] * 100:.2f}%)")
ax.set_title("PCA of genus-level log10 relative abundances")